## Based on the following paper: 
## XRFast a New Software Package for Processing of MA-XRF Datasets using Machine Leaning

Marc Vermeulen$^{1}$, Alicia McGeachy$^{1}$, Bingjie Xu$^{1}$, Henry Chopp$^{2}$, Aggelos Kataggelos$^{2}$, Rebecca Meyers$^{3}$, Matthias Alfeld$^{4}$, and Marc Walton$^{5,a,*}$

$^{1}$ Northwestern University / Art Institute of Chicago Center for Scientific Studies in the Arts (NU-ACCESS), 2145 Sheridan Road, Evanston, IL, United States

$^{2}$ Department of ECE, Northwestern University, Evanston, IL, United States

$^{3}$ National Museum of Mexican Art, 1852 W. 19th street, Chicago, IL, United States

$^{4}$ Delft University of Technology, Department of Materials Science and Engineering (MSE), 2628 CN Delft, Netherlands.

$^{5}$ M+, 38 Museum Drive, West Kowloon Cultural District, Hong Kong

$^{a}$ formerly: NU-ACCESS

$^{*}$ Corresponding author: marc.walton@mplus.org.hk

In [1]:
using XRFast
using DelimitedFiles 
using HDF5
using Gtk
using Statistics
using DelimitedFiles
using GLMakie

### Import XRF data and transform in appropriate format
Ideally, the format should be HDF5 but can also be ascii, tiff, etc.<br>

><ins>User input</ins>: Select the XRF file in cell 1. Run all other cells

In [2]:
filename = open_dialog("My Open dialog")

"/Users/homelyp/Desktop/Thoma_2002_6_fullscan_shift.HDF5"

In [3]:
root=splitext(filename)[1]

"/Users/homelyp/Desktop/Thoma_2002_6_fullscan_shift"

In [4]:
XRF = h5read(filename, "XRF")
XRF_spectra = XRF["Spectra"];

In [5]:
size(XRF_spectra)

(4096, 544, 412)

Flatten data into vector, "d"

In [6]:
d = reshape(XRF_spectra, size(XRF_spectra)[1], size(XRF_spectra)[2]*size(XRF_spectra)[3])
d = Float64.(d);

### Select desired number of endmembers ("atoms") from LLS transformed data and create initial dictionary 
The default value for the number of atoms is set at 20.

><ins>User input</ins>: Input the desired number of atoms (always better to over-estimate the number of atoms) <br> Default value is set to <b>20</b>.

In [7]:
Atoms_number = 20
@time D = initial_dictionary(d',
    Atoms = Atoms_number);

  0.075478 seconds (194.48 k allocations: 14.243 MiB, 97.61% compilation time)


### Perform dictionary learning to recover D

In this step, with each iteration a random 5% selection of A is used to optimize D.

><ins>User input</ins>: Select the number of iterations spent in outer loop (no_It_Sub) and the sparsity level (sparsity). <br> Default values are set to <b>30</b> for the "no_It_Sub" and <b>0.85</b> for the "sparsity".


In [8]:
no_It_Sub = 30
sparsity = 0.85

0.85

In [9]:
@time D = approx_nn_dictionary_learning(D, #initial dictionary
    d', #observations
    noIt_Sub = no_It_Sub, #iterations spent in outer loop, subsampling matrix L by 5%
    Lambda = sparsity, #Sparsity "level". Returns weights above a quantile (e.g., 0.5 = weights above the median value)
    );

Optimizing Dictionary...100%|███████████████████████████| Time: 0:10:04


613.157247 seconds (81.07 M allocations: 439.250 GiB, 8.96% gc time, 1.56% compilation time)


### Find $x$ for whole dataset

In [10]:
@time X_Sparse = nnls(d, D, sparsity);

291.488616 seconds (33.59 M allocations: 166.005 GiB, 1.33% gc time)


### Calculate Denoised Cube

In [11]:
Denoised = D*X_Sparse;

### Calulate similarity between spectra
The mean spectra of the output clusters along with the associated standard deviations (SD) per channel are then compared to make sure that the clusters are unique and does not require merging. Uniqueness is defined to be a difference in at least one channel that exceeds 3 SD.

In [12]:
D_mean = mean(D, dims=2);
D_std = std(D, dims=2);
D_similarity = abs.(D .- D_mean) .- 3 * D_std;
D_maxs = findmax(D_similarity, dims = 1);

### Interactive Plot
It will open in a <i>Makie</i> pop-up window <br>

<ins>Left:</ins> Atom's XRF spectrum. <br> <ins>Middle:</ins> Atom's sparse representation. <br> <ins>Right:</ins> Uniqueness index. Larger values indicate uniqueness of the atom.

In [13]:
fig = Figure()

ax1 = Axis(fig[1, 1])
ax2 = Axis(fig[1, 2])
ax3 = Axis(fig[1, 3])

sl_n = Slider(fig[2, 1], range = 1: Atoms_number, startvalue = 1) # all n numbers in slider

labeltext = lift(sl_n.value) do n
    string(Int(n))
end
Label(fig[2, 1], labeltext, tellwidth = false)

# Plot D (spectrums)

d = lift(sl_n.value) do n # change spectrum for each n
   D[:, n]
end

channels = range(1, size(XRF_spectra)[1], length=size(XRF_spectra)[1])

limits!(ax1, 1, size(XRF_spectra)[1], 0, maximum(D)) # axes limit
lines!(ax1, channels, d)

# PLot X_Sparse

xs = lift(sl_n.value) do n
   reshape(X_Sparse[n, :], size(XRF_spectra)[2], size(XRF_spectra)[3])
end

heatmap!(ax2, xs)

# Plot similarity for all spectrums

N = range(1, Atoms_number, length=Atoms_number) # total n number

Similarity = lift(sl_n.value) do n
    barplot!(ax3, N, D_maxs[1][:], width = step(N), strokewidth = 1, color = ifelse.(1:Atoms_number .== n, :red, :blue))
end

display(fig)

GLMakie.Screen(...)

### Save Sparse Representation Images
Save the sparse representation images (distribution maps) as .csv "text images". <br> 
<ul>
<li>The file will be saved in the same folder as the original datafile.</li>
    <li>The naming will include the number of iterations spent in outer loop (no_It_Sub) and the sparsity level (sparsity) defined previously, as well as the size of the original data cube, required to recreate the image.</li>
</ul>

Images can be recreated in by following the following steps: <br> 
<ol>
    <li><b>Import data</b>: File/Import/text Image...<br>
    <i>Select the desired .csv file</i></li>
    <li><b>Create a stack</b>: Image/Stacks/Tools/Montage to Stack...<br>
    <ins>Columns</ins>: last number of the filename, before the .csv <br>
    <ins>Rows</ins>: 1 <br>
    <ins>Border width</ins>: 0</li>
    <li><b>Transform compressed stack into stack of images</b>: Image/Stacks/Reslice[/]... (or Shortcut Stk/Reslice[/]...) on the <i>Stack</i> window created</li>
    <ins>Output spacing (pixels)</ins>: 1.000 <br>
    <ins>Start at</ins>: Top <br>
    <ins>Avoid interoplation</ins> should be checked <br>
    <i>Flip vertically</i> and <i>Rotate 90 degrees</i> can be checked or unchecked depending on the orientation of the original data. 
    <li><b>Save as a set of individual images</b>: File/Save As/Image Sequence... <br>
     <ins>Dir.</ins>: Select the folder where images will be saved <br>
        <ins>Format</ins>: select the desired format (TIFF is the default option) <br>
        <ins>Name</ins>: Indicate the root name of the images + EM (for endmember, recommended)<br>
        <ins>Start at</ins>: Indicate 1 (this correspond to the first endmember, which is the first slice)<br>
        <ins>Digits (1-8)</ins>: we recommend 2. This will save each image as EM01, EM02, [...] EM20).
</ol>

In [15]:
file2save=string(root,"_Sparse1","_OuterLoop_",no_It_Sub,"_Lambda_",sparsity,"_",size(XRF_spectra)[2],"x",size(XRF_spectra)[3],".csv")

"C:\\Users\\marcv\\Desktop\\Thoma_2002_6_fullscan_shift_Sparse1_OuterLoop_30_Lambda_0.85_544x412.csv"

In [16]:
writedlm(file2save,  X_Sparse, ',')

### Save XRF spectra associated with the the sparse representation images
XRF spectra can be recreated using Excel or any other spreadsheet/data analysis software (Origin, Igor, etc...) <br> 
<ul>
<li>The file will be saved in the same folder as the original datafile and Sparse representation .csv file.</li>
    <li>The naming will include the number of iterations spent in outer loop (no_It_Sub) and the sparsity level (sparsity) defined previously.</li>
</ul>

In [17]:
file2save2=string(root,"_Spectra_OuterLoop_",no_It_Sub,"_Lambda_",sparsity,".csv")

"C:\\Users\\marcv\\Desktop\\Thoma_2002_6_fullscan_shift_Spectra_OuterLoop_30_Lambda_0.85.csv"

In [18]:
writedlm(file2save2, D, ',')